In [1]:
/pwd

u'/Users/gunnarkleemann/Dropbox/coursework/BerkeleyDataSciences_8_11/BerkeleyCODE/W209code/GenomeWeb/PrePrc'

In [2]:
if __name__ == '__main__':

## dictionaries for recoding 
# for clinical significance
    ClinSigDict={0:99,1:8,2:1,3:2,4:-1,5:-2,6:6,7:7,255:0}
    
    #DisNmDict={}


##FUNCTIONS


########################################

# Melt data on column names list to serve as Id and value (melted) vars
# remove cols with zero values 
def Melton(splitCol, data):
    id_vars=data.columns.values.tolist()[0:splitCol]
    value_vars=data.columns.values.tolist()[splitCol:data.shape[1]]
    Melted=pd.melt(data, id_vars, value_vars)
    print 'lines before empty cell strip:' + str(len(Melted))
    Meltedfilt=Melted[Melted['value']==1]
    print 'lines After empty cell strip:' + str(len(Meltedfilt))
    Meltedfilt.to_csv('genomicDt.csv', sep='\t', encoding='utf-8', index=False)
    return Meltedfilt

########################################

#MAKE COLUMN NAMES UNIFORM # fix column names,
def fixDATA(df, useAlt):
    if '#CHROM' in df.columns:
        df.rename(columns={'#CHROM': 'CHROM'}, inplace=True)

    # remove the mutation address column if it is already there
    if 'Mut' in df.columns:
        df.drop(["Mut"], axis=1, inplace=True)

    #make all chrom titles uniform
    try:
        if len(df["CHROM"][1]) > 4:
            df["CHROM"] = df["CHROM"].apply(lambda x: x[3:len(x)])
    except:
        print "chr no len"

    # make mutation address column
    chr = df["CHROM"].map(str)
    #print df["POS"]
    pos = df["POS"].map(str)

    print useAlt

    if useAlt == 'yes':
        alt = df["ALT"].map(str)
        df['Mut'] = 'chr' + chr + '-' + pos + '-' + alt
        print 'added alt'
    else:
        df['Mut'] = 'chr' + chr + '-' + pos
        print 'added NO alt'
    return df

########################################

def FirstItem(str, sep):
    Item=str.split(sep)[0]
    return Item

#############GetInfo, Get 1 datum from info field#######################

### df=FiltList # this is the dataframe
### requires that the queried column is called'INFO' 
### fieldNm ='GENEINFO' #This is the field in the that we want to capture#
### ColNm = 'gene' #This is the name of column to put data into

#THE INFO FIELDS ARE NOT IN CONSTANT PLACES
#to find the right field, cycle through the elements of each list - the field changes position!
#NOTE: the info fields can contain repeat fields
#For the sake of this build I am taking the first
#Info = FiltList.INFO_y[n].split(';') # Parse Info data
#InfoNm needs to be dynamically assigned

def GetInfo(df, fieldNm, ColNm, InfoNm):    
    for n in range(len(df) - 1):   # for each line get the info
        Info = df.INFO_y[n].split(';') 
        Field = 'NotListed' # assume it is not there until its found
        for p in range(len(Info)): # cycle through info field for correct tag
            if Info[p][0:len(fieldNm)] == fieldNm: # parse data and put in the array
                str1=Info[p][len(fieldNm)+1:len(Info[p])]
                str2=FirstItem(str1, ':') #Remove number and take first item
                try:
                    Field=FirstItem(str2, '|')
                except:
                    pass
                try:
                    Field=FirstItem(str2, ',')
                except:
                    pass
            else:
                next
        df.loc[n, ColNm] = Field   
    return df

########################################




#Extract and clean the Clincal severity (CLNSIG), 
#and gene info (GENINFO) from the INFO columns
#InfoNm needs to be dynamically assigned
def GetClinDt(FiltList, InfoNm):
    for n in range(len(FiltList) - 1):
            Info = FiltList.INFO_y[n].split(';') # Parse Info data
            FiltList.loc[n, ('SnpDB')] = 'http://www.ncbi.nlm.nih.gov/snp/' + Info[0][3:len(Info[0])]  #gen the dbSNP link
            FiltList.loc[n, ('RSnum')] = Info[0][3:len(Info[0])]

    FiltList=GetInfo(FiltList, 'CLNSIG', 'ClinSig',InfoNm)  
    FiltList=GetInfo(FiltList, 'GENEINFO', 'Gene',InfoNm)    
    FiltList=GetInfo(FiltList, 'CLNDBN', 'Disease', InfoNm)                  
    return FiltList
        
# Transform the signficance values
def recode(SignifVal, RecodeDict):
    # for a tricolor (and black) scale...
    #0 - Uncertain significance (keep 0 middle value)
    #1 - not provided, (recode 255 black)
    #2 - Benign, (recode 1)
    #3 - Likely benign, (recode 2)
    #4 - Likely pathogenic,  (recode -1)
    #5 - Pathogenic, (recode -2)
    #6 - drug response(?)
    #7 - histocompatibility(?)
    #255 - "other" recode (0) unknown
    #RecodeDict={0:0,1:255,2:1,3:2,4:-1,5:-2,6:6,7:7} # pass this into function
    recodeVal=RecodeDict[SignifVal]
    return (recodeVal)
    

In [3]:
#IMPORT THE DATA - this method pulls lines from the s3 files. let try something 
#easier for now
#import numpy as np
#import pandas as pd
#import urllib


## these files are also on S3
## https://s3-us-west-2.amazonaws.com/209genomics/20130606_sample_info2.csv
## https://s3-us-west-2.amazonaws.com/209genomics/ALL.chrMT.phase3_callmom.20130502.genotypes.vcf.txt
## https://s3-us-west-2.amazonaws.com/209genomics/clinvar_20150804.vcf.txt


#opener = urllib.URLopener()
#myurl = "https://s3-us-west-2.amazonaws.com/209genomics/20130606_sample_info2.csv"
#myfile = opener.open(myurl)
#SampleInfo = myfile.read()
#SampleInfo=pd.read_table(SampleInfo, sep=',')
#print SampleInfo



In [4]:
import numpy as np
import pandas as pd
import urllib

Genomes='ALL.chrMT.phase3_callmom.20130502.genotypes.vcf.txt'
#MutRef='common_and_clinical.vcf.txt' # this one has no Mitochondrial X and Y data
SampleInfo='20130606_sample_info2.csv'
MutRef2='clinvar_20150804.vcf.txt' # this one HAS Mitochondrial X and Y data

SkipCt1=9
hdCt2=55
hdCt3=66

SampleInfo=pd.read_table(SampleInfo, sep=',')
Genomes=pd.read_table(Genomes, skiprows=SkipCt1)
#MutRef=pd.read_table(MutRef, skiprows=hdCt2, low_memory=False)
MutRef2=pd.read_table(MutRef2, skiprows=hdCt3, low_memory=False)
# the data from the genome files is wide format they have to be melted to make this work

In [5]:
# melt the data and strip empty cells - 
GenomesMelt=Melton(9, Genomes)

lines before empty cell strip:9816716
lines After empty cell strip:89695


In [6]:
#fix the names and add the MUT colum to use for joining
useAlt='yes'

MutFile_fx = fixDATA(MutRef2, useAlt) # use this file for MT data
Genomes_fx = fixDATA(Genomes, useAlt)
GenomesMelt_fx = fixDATA(GenomesMelt, useAlt)

yes
added alt
yes
added alt
chr no len
yes
added alt


In [7]:
#We need to put Genomes_fx into a file suitable for clustering 
# row ID is variant, column ID is person 

# one format is .PCL file (preclustering) suitable for Java treeview to cluster. 
# a simpler format is for clustering in R or using
Genomes_fx.drop(['CHROM','QUAL','ALT','REF','FILTER','INFO','FORMAT','ID'], axis=1, inplace=True)
Genomes_fx.columns

Index([u'POS', u'HG00096', u'HG00097', u'HG00099', u'HG00100', u'HG00101',
       u'HG00102', u'HG00103', u'HG00105', u'HG00106', 
       ...
       u'NA21129', u'NA21130', u'NA21133', u'NA21135', u'NA21137', u'NA21141',
       u'NA21142', u'NA21143', u'NA21144', u'Mut'],
      dtype='object', length=2536)

In [8]:
#clean up the HEATMAP MATRIX file

cols = Genomes_fx.columns.tolist()

Pos=Genomes_fx.iloc[:,0:1]

cols = cols[-1:] + cols[:-1]# move the mutation column into the first position

Genomes_fx=Genomes_fx[cols]

Genomes_fx.drop(['POS'], axis=1, inplace=True)

#Genomes_fx=Pos + Genomes_fx 

Genomes_fx.head(2)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Mut,HG00096,HG00097,HG00099,HG00100,HG00101,HG00102,HG00103,HG00105,HG00106,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,chrMT-10-C,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,chrMT-16-T,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Use the fixed clinvar file (MutFile) to get to the disease, clinical significance and gene information
#MutFile_fx.to_csv('ClinVar_fx.txt')
MutFile_fx.head(2)

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,Mut
0,1,1014143,rs786201005,C,T,.,.,RS=786201005;RSPOS=1014143;dbSNPBuildID=144;SS...,chr1-1014143-T
1,1,1014316,rs672601345,C,CG,.,.,RS=672601345;RSPOS=1014319;dbSNPBuildID=142;SS...,chr1-1014316-CG


In [10]:
SampleInfo.head(1)

,Sample,Family ID,Population,Population Description,Gender,Relationship,Unexpected Parent/Child,Non Paternity,Siblings,Grandparents,Avuncular,Half Siblings,Unknown Second Order,Third Order,Other Comments
0,HG00096,HG00096,GBR,British in England and Scotland,male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
##FILTER FILES FOR KNOWN MUTATIONS
## Merge both INDIVIDUAL AND MATRIX data on the list of known mutations (ClivarList = MutFile_fx )
## This filter removes mutations that we have no information about and 
## puts the gene, clinical significance and gene data in register with the mutation (MUT) Column.

print 'lines before Clinvar filter:' + str(len(Genomes_fx))
#The heatmap data
FiltList = pd.merge(Genomes_fx, MutFile_fx, on=['Mut'], how='inner')
#The individual genome summary
FiltListMelt = pd.merge(GenomesMelt_fx, MutFile_fx, on=['Mut'], how='inner')
print 'lines After ClinVar filter:' + str(len(FiltList))
print str(len(np.unique(FiltList['Mut']))) + ' Unique Mutations'
#print ('\n')
#print 'heatmap (FiltList)'
#print ('\n')
#print FiltList.head(3)
#print ('\n')
#print 'Individual summary (FiltListMelt)'
#print ('\n')
#print FiltListMelt.head(3)

lines before Clinvar filter:3874
lines After ClinVar filter:111
111 Unique Mutations


In [12]:
## WE MAY OPT TO REMOVE COLUMNS/ ROWS where the "mutation" 
## is found in most patients and therefore not a mutation.
FiltList.shape

(111, 2543)

In [13]:
#Merge the sample info with INDIVUDUAL GENOME SUMMARY
#list on the genome(variable/Sample) feature and clean up

FiltListMelt.rename(columns={'variable':'Sample'}, inplace=True)
FiltListMelt = pd.merge(FiltListMelt, SampleInfo, on=['Sample'], how='inner')
FiltListMelt.drop(FiltListMelt.columns[24:len(FiltListMelt.columns)], axis=1, inplace= True)

#FiltList.drop(FiltList.columns[[2,17,18]], axis=1, inplace= True)
#FiltList.drop(FiltList.columns[[5,7,9]], axis=1, inplace= True)
FiltListMelt.drop(['ID_x','QUAL_x', 'FILTER_x', 'FORMAT','ID_y','QUAL_y', 'FILTER_y'], axis=1, inplace= True)

print FiltListMelt.columns
print FiltListMelt.shape

Index([u'CHROM_x', u'POS_x', u'REF_x', u'ALT_x', u'INFO_x', u'Sample',
       u'value', u'Mut', u'CHROM_y', u'POS_y', u'REF_y', u'ALT_y', u'INFO_y',
       u'Family ID', u'Population', u'Population Description', u'Gender'],
      dtype='object')
(10424, 17)


In [14]:
##FOR THE INDIVIDUALS FILE - ADD the DISEASE DATA AND SAVE
#FiltListMelt.to_csv('1000genomesMtData')
FiltList2=GetClinDt(FiltListMelt,'INFO_y')

#OUTPUT THE INDIVIDUAL DATA
out='INDIVgenomicDt'
datasrc='Mt'
Align='ALL.chrMT.phase3_callmom.20130502.genotypes.vcf.txt'
ref='common_and_clinical.vcf.txt'
fileNm = out + '_' + datasrc + '.csv'
FiltList2.to_csv(fileNm)

print FiltList2.columns
print FiltList2.shape
print FiltList.head()

Index([u'CHROM_x', u'POS_x', u'REF_x', u'ALT_x', u'INFO_x', u'Sample',
       u'value', u'Mut', u'CHROM_y', u'POS_y', u'REF_y', u'ALT_y', u'INFO_y',
       u'Family ID', u'Population', u'Population Description', u'Gender',
       u'SnpDB', u'RSnum', u'ClinSig', u'Gene', u'Disease'],
      dtype='object')
(10424, 22)
             Mut  HG00096  HG00097  HG00099  HG00100  HG00101  HG00102  \
0    chrMT-827-G        0        0        0        0        0        0   
1    chrMT-951-A        0        0        0        0        0        0   
2  chrMT-961-C,G        0        0        0        0        0        0   
3    chrMT-980-C        0        0        0        0        0        0   
4    chrMT-990-C        0        0        0        0        0        0   

   HG00103  HG00105  HG00106  \
0        0        0        0   
1        0        0        0   
2        0        0        0   
3        0        0        0   
4        0        0        0   

                         ...                

In [15]:
##Add new column to Individual file to recode DISEASE NAMES to simple IDS.

FiltList2['DiseaseSumm']=FiltList2['Disease']
FiltList2.head()


dlst=FiltList2.groupby('Disease')
Dlist=dlst.Disease.unique()
Dlist.to_csv("diseaseList.csv")
Dlist

## lets make a dictionary to be used to simpilfy terms##
diseaseDict ={'Aminoglycoside-induced_deafness|Deafness\\x2c_nonsyndromic_sensorineural\\x2c_mitochondrial|Auditory_neuropathy|not_specified': 'Deafness',  
'Brain_pseudoatrophy\\x2c_reversible\\x2c_valproate-induced\\x2c_susceptibility_to': 'Brain_pseudoatrophy',
'Cardiomyopathy\\x2c_idiopathic_dilated\\x2c_mitochondrial|Cardiomyopathy\\x2c_hypertrophic\\x2c_mitochondrial': 'Cardiomyopathy',
'Cardiomyopathy\\x2c_mitochondrial': 'Cardiomyopathy',
'Cytochrome-c_oxidase_deficiency': 'Cytochrome-c deficiency',
'Deafness\\x2c_nonsyndromic_sensorineural\\x2c_mitochondrial': 'Deafness',
'Deafness\\x2c_nonsyndromic_sensorineural\\x2c_mitochondrial|Aminoglycoside-induced_deafness': 'Deafness',
'Familial_cancer_of_breast': 'Breast cancer',
'Familial_colorectal_cancer': 'colorectal cancer',
'Focal_segmental_glomerulosclerosis_and_dilated_cardiomyopathy': 'Glomerulosclerosis and Cardiomyopathy',
'Juvenile_myopathy\\x2c_encephalopathy\\x2c_lactic_acidosis_AND_stroke': 'Juvenile_myopathy...',
"Leber's_optic_atrophy": 'Lebers_optic_atrophy',
"Leber's_optic_atrophy|Aminoglycoside-induced_deafness|Deafness\\x2c_nonsyndromic_sensorineural\\x2c_mitochondrial": 'Lebers_optic_atrophy',
"Leber_hereditary_optic_neuropathy_with_dystonia|Leber's_optic_atrophy": 'Lebers_optic_atrophy',
"Leigh's_disease": 'Leighs_disease',
'Neoplasm_of_ovary': 'Neoplasm_of_ovary',
'Obesity': 'Obesity',
'Parkinson_disease\\x2c_resistance_to': 'Parkinson_disease',
'Parkinson_disease_6\\x2c_autosomal_recessive_early-onset': 'Parkinson_disease',
'Primary_familial_hypertrophic_cardiomyopathy|Deafness\\x2c_nonsyndromic_sensorineural\\x2c_mitochondrial': 'cardiomyopathy',
'Sensorineural_deafness_and_migraine': 'Deafness',
'Sudden_death': 'Sudden death',
'Sudden_infant_death_syndrome': 'SIDS',
'not_specified': 'not specified'}

## now replace the disease names in the new columns with the summary names

for n in range(len(FiltList2)):
    try :
        FiltList2.iloc[n, 22]=diseaseDict[FiltList2.iloc[n, 21]] 
    except:
        print 'Found a NaN on line' + str(n)
        FiltList2.iloc[n, 22]='NaN'


Found a NaN on line10423


In [16]:
##NOW FOCUS ON THE HEATMAP MATRIX## 
# Recode called mutations with thier clinical significance and 
# EITHER APPEND OR generate a second dataframe with disease and gene names. 

# The internal values are clincal significance and should be coded more intuitively
# ##INFO=<ID=CLNSIG,Number=.,Type=String,Description="Variant Clinical Significance, 

# for a tricolor scale...
#0 - Uncertain significance (keep 0 middle value)
#1 - not provided, (recode 255 black)
#2 - Benign, (recode 1)
#3 - Likely benign, (recode 2)
#4 - Likely pathogenic,  (recode -1)
#5 - Pathogenic, (recode -2)
#6 - drug response(?)
#7 - histocompatibility(?) 
# need to recode the non-muation sites 255 - Just Drop these 
## have 6, 7 and 255 been stripped by the fix data function?

In [17]:
SampleInfo.tail()

,Sample,Family ID,Population,Population Description,Gender,Relationship,Unexpected Parent/Child,Non Paternity,Siblings,Grandparents,Avuncular,Half Siblings,Unknown Second Order,Third Order,Other Comments
3495,NA21137,NA21137,GIH,"Gujarati Indian in Houston,TX",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3496,NA21141,NA21141,GIH,"Gujarati Indian in Houston,TX",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3497,NA21142,NA21142,GIH,"Gujarati Indian in Houston,TX",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3498,NA21143,NA21143,GIH,"Gujarati Indian in Houston,TX",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3499,NA21144,NA21144,GIH,"Gujarati Indian in Houston,TX",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
## FIX THE MATRIX 


##split out the HEATMAP data, retain mutation 'mut' column
Mut_Matrix=FiltList.iloc[:,0:2535]

    #Mut_Matrix.drop('POS_x', axis=1, inplace=True)    
Mut_nparray = Mut_Matrix.as_matrix() # make a numpy array for searching
print Mut_nparray.shape
print Mut_nparray[1:5,]

(111, 2535)
[['chrMT-951-A' 0 0 ..., 0 0 0]
 ['chrMT-961-C,G' 0 0 ..., 0 0 0]
 ['chrMT-980-C' 0 0 ..., 0 0 0]
 ['chrMT-990-C' 0 0 ..., 0 0 0]]


In [19]:
# FIND THE INDEX OF THE MUTATIONS (non-zeros) and the WT sites (zeros)) 
# transpose is used to coerce the indexes into a usable shape
#NonZeroIdx=np.transpose(np.nonzero(Mut_nparray[:,1:Mut_nparray.shape[1]]))
#ZeroIdx=np.transpose(np.where(Mut_nparray == 0))
#adjust for the header row.
#NonZeroIdx[:,1] = NonZeroIdx[:,1]+1
#ZeroIdx[:,1] = ZeroIdx[:,1]+1
#print NonZeroIdx[1:5,:]
#print ZeroIdx[1:5,:]
#print NonZeroIdx[:,1]

In [20]:
## RECODE the matrix replacing mutations with thier severity score
## keep a copy of the matrix for comparison and to indicate haploid(1) or diploid(2)
ClinSigDict={0:99,1:8,2:1,3:2,4:-1,5:-2,6:6,7:7,255:0}
Mut_MatrixRECODE=Mut_Matrix.replace(ClinSigDict)

In [21]:
Mut_MatrixRECODE=Mut_MatrixRECODE.transpose()
Mut_Matrix=Mut_Matrix.transpose()

In [22]:
## RECODE the matrix replacing mutations with thier severity score
## keep a copy of the matrix for comparison and to indicate haploid(1) or diploid(2)


# lookup the mutation - Get the severity score and Place it in the matrix 
#for ii in range(len(NonZeroIdx)):
#    MutID=Mut_nparray[NonZeroIdx[ii][0],0] # identify the mutation
#    #print MutID + '_' + str(NonZeroIdx[z][0]) + "_" + str((NonZeroIdx[z][1])+1)
#    row=np.where(FiltListMelt['Mut']==MutID)[0][0]
#    ClinSig=int(FirstItem(FiltListMelt.iloc[row,19], '|'))
#    #print ClinSig
#    ClinSig=recode(ClinSig, ClinSigDict)
#    #Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]=ClinSig
#    if (Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]) == 0: # none of these positions should be 0
#        print 'replacing' + str(Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]) + 'with' + str(ClinSig)
#        print "cell" + str(NonZeroIdx[ii][1]) + 'and '+  str(NonZeroIdx[ii][0])
    
#    try:
#        Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]=ClinSig
#    except:
#        print 'replacing' + str(Mut_Matrix.iloc[NonZeroIdx[ii][0],(NonZeroIdx[ii][1])]) + 'with' + str(ClinSig)
#        print "failed at cell" + str(NonZeroIdx[ii][1]) + 'x cell '+  str(NonZeroIdx[ii][0])
        


In [23]:
# Lookup WT alleles get position and code with 255
#for j in range(len(ZeroIdx)):
    #MutID=Mut_nparray[ZeroIdx[j][0],0]
    #print MutID + '_' + str(NonZeroIdx[z][0]) + "_" + str((NonZeroIdx[z][1])+1)
    #row=np.where(FiltListMelt['Mut']==MutID)[0][0]  
    
    # none of these positions should be non-0
#    if (Mut_Matrix.iloc[ZeroIdx[ii][0],(ZeroIdx[ii][1])]) != 0: 
#        print 'replacing' + str(Mut_Matrix.iloc[ZeroIdx[ii][0],(ZeroIdx[ii][1])]) + 'with' + str(ClinSig)
#        print "cell" + str(ZeroIdx[ii][1]) + 'and '+  str(ZeroIdx[ii][0])
    
#    try:
#        Mut_Matrix.iloc[ZeroIdx[j][0],(ZeroIdx[j][1])]=255 #fill all of the WT positions
        #print Mut_Matrix.iloc[ZeroIdx[j][0],(ZeroIdx[j][1])]
#    except:
        # it is missing the last row...
#        print ZeroIdx[j][0]
#        print ZeroIdx[j][1]
        #print j

In [24]:
SampleID

NameError: name 'SampleID' is not defined

In [ ]:
## EXTRACT KEYS AND MATCH WITH PERTINENT DATA


### NEED TO ADD DISESES AND GENES HEE
#MutNames.columns
#MutNames=pd.merge(MutNames, SampleInfo, on=['Sample'], how='inner')
#SampleInfo
#THIS WILL BE USED TO GET THE GENE/DISEASE INFOR
####>MutNames=pd.DataFrame(Mut_Matrix.iloc[0,:], columns=['Mut'])
####>MutNames=pd.merge(MutNames, SampleInfo, on=['Sample'], how='inner') 
####>MutNames = pd.merge(MutNames, MutFile_fx, on=['Mut'], how='inner')
####>to_csv()

#print MutNames
SampleID=pd.DataFrame(Mut_Matrix.index, columns =['Sample'])
SampleID=pd.merge(SampleID, SampleInfo, on=['Sample'], how='inner') 
SampleID=SampleID.iloc[:,1:5]
SampleID['PopCode']=SampleID['Population']
SampleID['PopCode']=SampleID['PopCode'].replace(['GBR', 'FIN', 'CHS', 'PUR', 'CDX', 'CLM', 'IBS', 'PEL', 'PJL',
       'KHV', 'ACB', 'GWD', 'ESN', 'BEB', 'MSL', 'STU', 'ITU', 'CEU',
       'YRI', 'CHB', 'JPT', 'LWK', 'ASW', 'MXL', 'TSI', 'GIH'], ['1','2','3','4','5','6','7','8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18',
       '19', '20', '21', '22', '23', '24', '25', '26'])

SampleID.to_csv('SampleID.csv', sep='\t', encoding='utf-8', index=False)

#Mut_MatrixRECODE.head()
# attatch the population data to reference array MutMatrix
#Mut_Matrix.columns
#={'':'Sample'}, inplace=True)
#Mut_Matrix.head()

In [121]:
SampleID

,Family ID,Population,Population Description,Gender,PopCode
0,HG00096,GBR,British in England and Scotland,male,1
1,HG00097,GBR,British in England and Scotland,female,1
2,HG00099,GBR,British in England and Scotland,female,1
3,HG00100,GBR,British in England and Scotland,female,1
4,HG00101,GBR,British in England and Scotland,male,1
5,HG00102,GBR,British in England and Scotland,female,1
6,HG00103,GBR,British in England and Scotland,male,1
7,HG00105,GBR,British in England and Scotland,male,1
8,HG00106,GBR,British in England and Scotland,female,1
9,HG00107,GBR,British in England and Scotland,male,1


In [53]:
## transpose matrix to put genotype on each row and add gender and populaton data for each sample

Mut_MatrixRECODE.head()

# attatch the population data to reference array MutMatrix
Mut_Matrix.rename(columns={'':'Sample'}, inplace=True)
Mut_Matrix.head()
Mut_Matrix2=pd.merge(Mut_Matrix, SampleInfo, on=['Sample'], how='inner')
Mut_Matrix.head()

KeyError: 'Sample'

In [55]:
print Mut_Matrix['Mut']

KeyError: 'Mut'

In [ ]:
Mut_Matrix.rename(columns = {'':'Sample'}, inplace = True)
Mut_Matrix.head()

## add enthnicity column
## add disease column

#add
#FiltListMelt = pd.merge(FiltListMelt, SampleInfo['Populaton'], on=['Sample'], how='inner')


In [196]:
Mut_Matrix.rename(columns = {'':'Sample'}, inplace = True)
Mut_Matrix.tail()

NameError: name 'Mut_Matrix' is not defined

In [ ]:
#Transpose to make individual genomes listed on rows rather than columns
Mut_Matrix=Mut_Matrix.transpose()
        #add ethnicity
#FiltListMelt = pd.merge(Mut_Matrix, SampleInfo['Population'], on=['Sample'], how='inner')
#Output the data
out='MatrixGenomDt'
datasrc='Mt'
fileNm = out + '_' + datasrc + '.csv'
Mut_Matrix.to_csv(fileNm, sep="\t", index_label="False", index="False")

In [119]:
###format data - http://blog.nextgenetics.net/?e=44###

In [153]:
FiltListMelt.head()

,CHROM_x,POS_x,REF_x,ALT_x,INFO_x,Sample,value,Mut,CHROM_y,POS_y,...,ALT_y,INFO_y,Family ID,Population,Population Description,Gender,SnpDB,RSnum,Clinical_Significance,gene
0,MT,1438,A,G,VT=S;AC=2419,HG00096,1,chrMT-1438-G,MT,1438,...,G,RS=2001030;RSPOS=1438;dbSNPBuildID=92;SSR=0;SA...,HG00096,GBR,British in England and Scotland,male,http://www.ncbi.nlm.nih.gov/snp/2001030,2001030,2,NaN
1,MT,15326,A,G,VT=S;AC=2521,HG00096,1,chrMT-15326-G,MT,15326,...,G,RS=2853508;RSPOS=15326;dbSNPBuildID=100;SSR=0;...,HG00096,GBR,British in England and Scotland,male,http://www.ncbi.nlm.nih.gov/snp/2853508,2853508,4,CYTB
2,MT,1438,A,G,VT=S;AC=2419,HG00097,1,chrMT-1438-G,MT,1438,...,G,RS=2001030;RSPOS=1438;dbSNPBuildID=92;SSR=0;SA...,HG00097,GBR,British in England and Scotland,female,http://www.ncbi.nlm.nih.gov/snp/2001030,2001030,2,NaN
3,MT,15326,A,G,VT=S;AC=2521,HG00097,1,chrMT-15326-G,MT,15326,...,G,RS=2853508;RSPOS=15326;dbSNPBuildID=100;SSR=0;...,HG00097,GBR,British in England and Scotland,female,http://www.ncbi.nlm.nih.gov/snp/2853508,2853508,4,CYTB
4,MT,4216,T,C,VT=S;AC=139,HG00097,1,chrMT-4216-C,MT,4216,...,C,RS=1599988;RSPOS=4216;dbSNPBuildID=88;SSR=1;SA...,HG00097,GBR,British in England and Scotland,female,http://www.ncbi.nlm.nih.gov/snp/1599988,1599988,0,ND1
